In [8]:
import pandas as pd

In [55]:
deodorants_df_0 = pd.read_csv("../clean_data/deodorants_0.csv", parse_dates=['fecha_trans', 'fecha_desde', 'fecha_hasta'])
deodorants_df_1 = pd.read_csv("../clean_data/deodorants_1.csv", parse_dates=['fecha_trans', 'fecha_desde', 'fecha_hasta'])
deodorants_df_2 = pd.read_csv("../clean_data/deodorants_2.csv", parse_dates=['fecha_trans', 'fecha_desde', 'fecha_hasta'])
deodorants_df_3 = pd.read_csv("../clean_data/deodorants_3.csv", parse_dates=['fecha_trans', 'fecha_desde', 'fecha_hasta'])
deodorants_df_4 = pd.read_csv("../clean_data/deodorants_4.csv", parse_dates=['fecha_trans', 'fecha_desde', 'fecha_hasta'])
deodorants_df_5 = pd.read_csv("../clean_data/deodorants_5.csv", parse_dates=['fecha_trans', 'fecha_desde', 'fecha_hasta'])
deodorants_df_6 = pd.read_csv("../clean_data/deodorants_6.csv", parse_dates=['fecha_trans', 'fecha_desde', 'fecha_hasta'])
deodorants_df_7 = pd.read_csv("../clean_data/deodorants_7.csv", parse_dates=['fecha_trans', 'fecha_desde', 'fecha_hasta'])
deodorants_df_8 = pd.read_csv("../clean_data/deodorants_8.csv", parse_dates=['fecha_trans', 'fecha_desde', 'fecha_hasta'])
deodorants_df_9 = pd.read_csv("../clean_data/deodorants_9.csv", parse_dates=['fecha_trans', 'fecha_desde', 'fecha_hasta'])
deodorants_df_10 = pd.read_csv("../clean_data/deodorants_10.csv", parse_dates=['fecha_trans', 'fecha_desde', 'fecha_hasta'])
deodorants_df_11 = pd.read_csv("../clean_data/deodorants_11.csv", parse_dates=['fecha_trans', 'fecha_desde', 'fecha_hasta'])

In [73]:
deodorants_df = pd.concat([deodorants_df_0, deodorants_df_1,deodorants_df_2, deodorants_df_3,
                           deodorants_df_4, deodorants_df_5, deodorants_df_6,
                           deodorants_df_7, deodorants_df_8, deodorants_df_9,
                           deodorants_df_10, deodorants_df_11], axis=0, ignore_index=True)

In [74]:
deodorants_df.duplicated().sum()

0

In [75]:
deodorants_df.shape

(8356436, 26)

In [80]:
deodorants_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8356436 entries, 0 to 8356435
Data columns (total 26 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   idb                     int64         
 1   id_producto             int64         
 2   fecha_trans             datetime64[ns]
 3   fecha_desde             datetime64[ns]
 4   fecha_hasta             datetime64[ns]
 5   stock_unidades          int64         
 6   venta_unidades          int64         
 7   precio_unitario         float64       
 8   sigue_prod_en_rol       int64         
 9   cd_abast                int64         
 10  mov_month               int64         
 11  mov_year                int64         
 12  mov_weekday             int64         
 13  ventas_estimadas_promo  float64       
 14  nro_prom                float64       
 15  duracion_promo          float64       
 16  id_cadena               int64         
 17  desc_cadena             object        
 18  de

In [ ]:
deodorants_df.shape

In [81]:
# create new column with 1 or 0 based on conditions
deodorants_df['tiene_promo'] = ((deodorants_df['nro_prom'].notnull()) & (deodorants_df['fecha_desde'].notnull()) & (deodorants_df['fecha_hasta'].notnull())).astype(int)

In [82]:
counts = deodorants_df.groupby('tiene_promo').size()
# print counts
print(counts)

tiene_promo
0    8220158
1     136278
dtype: int64


In [85]:
# filter dataframe where nro_promo is not null
filtered_df = deodorants_df[deodorants_df['nro_prom'].notnull()]

# group by producto and idb and count number of rows in each group
grouped_df = filtered_df.groupby(['id_producto', 'idb', 'id_cadena']).size().reset_index(name='count')

# print grouped dataframe
print(grouped_df)

      id_producto   idb  id_cadena  count
0          164146   285          2     16
1          164146  5034          9     10
2          164146  5201          9     40
3          164146  5202          9     50
4          164146  5204          9     50
...           ...   ...        ...    ...
5125       507528  9138          9      8
5126       507528  9234          4     15
5127       507528  9240          4     15
5128       507528  9692          4     15
5129       507528  9698          4     15

[5130 rows x 4 columns]


In [ ]:
deodorants_df.to_csv('../clean_data/deodorants_final_merged.csv', index=False)

In [86]:
deodorants_df.describe()

,idb,id_producto,stock_unidades,venta_unidades,precio_unitario,sigue_prod_en_rol,cd_abast,mov_month,mov_year,mov_weekday,ventas_estimadas_promo,nro_prom,duracion_promo,id_cadena,longitud,latitud,local,punto_dist,id_region,tiene_promo
count,8.356436e+06,8.356436e+06,8.356436e+06,8.356436e+06,8.356436e+06,8.356436e+06,8.356436e+06,8.356436e+06,8.356436e+06,8.356436e+06,136278.000000,136278.000000,136278.000000,8.356436e+06,8.356436e+06,8.356436e+06,8.356436e+06,8.356436e+06,7757988.0,8.356436e+06
mean,6.989773e+03,3.785508e+05,2.065137e+01,3.136310e-01,6.423475e+01,9.610200e-01,5.434609e+03,6.485025e+00,2.017666e+03,3.002266e+00,23.915526,477085.219096,9.107919,4.277160e+00,-5.872835e+01,-3.506354e+01,9.897948e-01,1.020519e-02,1.0,1.630815e-02
std,3.299906e+03,9.158304e+04,1.487243e+02,8.953579e-01,6.580358e+01,1.935474e-01,4.533852e+03,3.453803e+00,9.815391e-01,2.000099e+00,31.778576,39563.177966,3.569107,2.618480e+00,1.394684e+00,2.036040e+00,1.005039e-01,1.005039e-01,0.0,1.266578e-01
min,2.420000e+02,1.641460e+05,-5.420000e+02,-4.500000e+01,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,2.016000e+03,0.000000e+00,0.000000,425196.000000,3.000000,2.000000e+00,-6.749604e+01,-4.587265e+01,0.000000e+00,0.000000e+00,1.0,0.000000e+00
25%,5.208000e+03,2.929450e+05,6.000000e+00,0.000000e+00,2.251000e+01,1.000000e+00,0.000000e+00,3.000000e+00,2.017000e+03,1.000000e+00,4.000000,440560.000000,7.000000,2.000000e+00,-5.869611e+01,-3.498957e+01,1.000000e+00,0.000000e+00,1.0,0.000000e+00
50%,9.026000e+03,4.152680e+05,1.000000e+01,0.000000e+00,3.823000e+01,1.000000e+00,9.217000e+03,6.000000e+00,2.018000e+03,3.000000e+00,12.000000,473864.000000,7.000000,4.000000e+00,-5.846482e+01,-3.462789e+01,1.000000e+00,0.000000e+00,1.0,0.000000e+00
75%,9.159000e+03,4.583930e+05,1.700000e+01,0.000000e+00,7.673000e+01,1.000000e+00,9.217000e+03,1.000000e+01,2.018000e+03,5.000000e+00,30.000000,518153.000000,9.000000,4.000000e+00,-5.839790e+01,-3.457732e+01,1.000000e+00,0.000000e+00,1.0,0.000000e+00
max,9.977000e+03,5.113260e+05,1.621800e+04,6.120000e+02,4.238100e+02,1.000000e+00,9.217000e+03,1.200000e+01,2.019000e+03,6.000000e+00,300.000000,541066.000000,18.000000,9.000000e+00,-5.668353e+01,-2.914198e+01,1.000000e+00,1.000000e+00,1.0,1.000000e+00


In [89]:
# check for null or NaN values in each column
null_cols = deodorants_df.isnull().any()

# print columns with null or NaN values
print(null_cols[null_cols == True].index.tolist())

['fecha_desde', 'fecha_hasta', 'ventas_estimadas_promo', 'nro_prom', 'duracion_promo', 'desc_producto', 'id_region']


In [88]:
deodorants_df.drop('razon_social', axis=1, inplace=True)

In [105]:
# group by id_producto and check if any tiene_prom value is 1
products_with_promotion = deodorants_df.groupby(['id_producto'])['tiene_promo'].any()

# products_with_promotion now contains a boolean value for each id_producto indicating if that product has ever had a promotion (True) or not (False)
products_without_promotion = products_with_promotion[~products_with_promotion].index.tolist()

products_without_promotion

[238237,
 249049,
 385535,
 407845,
 443520,
 443521,
 443547,
 450083,
 454813,
 469286,
 485719,
 497936,
 507832,
 511324,
 511325,
 511326]